#### Imports

In [ ]:
import os
import re
import sys
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx

In [ ]:
sys.path.append('..')

In [ ]:
from src.utils.image_utils import load_czi_images, enhance_cell_image_contrast
from src.utils.plot_utils import show_3d_segmentation_overlay
from src.track import *

#### Functions

#### Inputs

In [ ]:
czi_file_path = '/home/dafei/data/MS2/gRNA2_12.03.25-st-13-II---.czi'
seg_maps_dir = '/home/dafei/output/MS2/3d_cell_segmentation/gRNA2_12.03.25-st-13-II---/masks'

In [ ]:
image_data = load_czi_images(czi_file_path)

In [ ]:
def extract_time_number(filename):
    """Extract time number from filename like z_stack_t5_seg_masks.npz"""
    try:
        # Find the pattern t{number}_
        match = re.search(r't(\d+)_', filename)
        return int(match.group(1)) if match else 0
    except:
        return 0

# Sort files by time number
files = sorted([os.path.join(seg_maps_dir, x) for x in os.listdir(seg_maps_dir) 
                if x.startswith('z_stack_t') and x.endswith('_seg_masks.npz')], 
               key=extract_time_number)

In [ ]:
files


In [ ]:
z_stack_seg_mask_t0 = np.load(files[0])['masks']
z_stack_t0 = image_data[0, 0, 1, :, :, :, 0]

In [ ]:
show_3d_segmentation_overlay(z_stack_t0, z_stack_seg_mask_t0)

In [ ]:
z_stack_seg_mask_t1 = np.load(files[1])['masks']
z_stack_t1 = image_data[0, 1, 1, :, :, :, 0]

In [ ]:
show_3d_segmentation_overlay(z_stack_t1, z_stack_seg_mask_t1)

In [ ]:
z_stack_seg_mask_t2 = np.load(files[2])['masks']
z_stack_t2 = image_data[0, 2, 1, :, :, :, 0]

In [ ]:
show_3d_segmentation_overlay(z_stack_t2, z_stack_seg_mask_t2)

Tracking cells using adjacency graph

In [ ]:
# Compute cell locations for the first frame
centers1 = get_cell_centers(z_stack_seg_mask_t0)
labels1 = np.unique(z_stack_seg_mask_t0)
g1 = compute_cell_location(centers=centers1, labels=labels1)

# Compute cell locations for the second frame
centers2 = get_cell_centers(z_stack_seg_mask_t1)
labels2 = np.unique(z_stack_seg_mask_t1)
g2 = compute_cell_location(centers=centers2, labels=labels2)

# Compute cell locations for the third frame
centers3 = get_cell_centers(z_stack_seg_mask_t2)
labels3 = np.unique(z_stack_seg_mask_t2)
g3 = compute_cell_location(centers=centers3, labels=labels3)

In [ ]:
matches_1_t0_0 = match_points_between_frames(g1, g2, z_stack_seg_mask_t0, z_stack_seg_mask_t1,distance_threshold=np.sqrt(3))
matches_2_t1_1 = match_points_between_frames(g2, g3, z_stack_seg_mask_t1, z_stack_seg_mask_t2,distance_threshold=np.sqrt(3))

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

for label_t_plus1 in range(1,449):
    label_t = matches_1_t0_0[label_t_plus1] if label_t_plus1 in matches_1_t0_0 else None
    if label_t is None:
        continue  # Skip if no match found
    # Get positions for the current label in both frames
    row_mask_t1 =centers2[:, 0] == label_t_plus1
    matching_rows1 = centers2[row_mask_t1]
    pos1 = matching_rows1[0][1:]

    row_mask_t2 = centers1[:, 0] == label_t
    matching_rows2 = centers1[row_mask_t2]
    pos2 = matching_rows2[0][1:]
    
    ax.scatter(*pos1, color='blue', label=f'Cell {label_t_plus1} in Frame 2',s=4)
    ax.scatter(*pos2, color='red', label=f'Cell {label_t} in Frame 1' if pos2 is not None else 'No match', s=4)
    ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], [pos1[2], pos2[2]], 'black', linewidth=1, linestyle='--' if pos2 is not None else 'None')
    ax.set_title('Cell Tracking Between Frames')
    ax.set_xlim([0, z_stack_seg_mask_t1.shape[1]])
    ax.set_ylim([0, z_stack_seg_mask_t1.shape[2]])
    ax.set_zlim([0, z_stack_seg_mask_t1.shape[0]])
#ax.legend()
ax.grid(True)
plt.show()

In [ ]:
label_t_plus1 = 1
label_t = matches_1_t0_0[label_t_plus1] if label_t_plus1 in matches_1_t0_0 else None

cell_mask_t_plus1 = (z_stack_seg_mask_t1 == label_t_plus1).astype(np.uint8)
cell_mask_t = (z_stack_seg_mask_t0 == label_t).astype(np.uint8)

show_3d_segmentation_overlay(z_stack_t1, cell_mask_t_plus1)
show_3d_segmentation_overlay(z_stack_t0, cell_mask_t)

In [ ]:
def find_key_for_last_tracklet_value_optimized(tracklets, matches_dict, tracklet_id):
    """
    Optimized version using next() with generator expression for early termination.
    """
    last_value = tracklets[tracklet_id][-1]
    
    # Use next() with generator for immediate return on first match
    return next((key for key, value in matches_dict.items() if value == last_value), -1)

In [ ]:
def create_tracklets(matches:list):
    matches_1_t0_0 = matches[0]
    #init tracklets
    tracklets = {}
    for i,(label_t_plus1, label_t) in enumerate(matches_1_t0_0.items()):
        tracklets[i] = [int(label_t),int(label_t_plus1)]
    for i in range(1, len(matches)):
        max_id = max(tracklets.keys())
        new_tracklets = {}

        # First, extend existing tracklets
        for id, labels in tracklets.items():
            key = find_key_for_last_tracklet_value_optimized(tracklets, matches_2_t1_1, id)
            if key != -1:  # Only append if a match was found
                labels.append(int(matches_2_t1_1[key]))

    # Then, create new tracklets for unmatched cells in matches_2_t1_1
    used_keys = set()
    for id, labels in tracklets.items():
        if len(labels) >= 2:  # Check if tracklet was extended
            used_keys.add(labels[-2])  # Add the second-to-last value (the key that was found)

    # Create new tracklets for unused matches
    for key, value in matches_2_t1_1.items():
        if key not in used_keys:
            max_id += 1
            new_tracklets[max_id] = [-1]*i + [int(key), int(value)]

    # Add new tracklets
    tracklets.update(new_tracklets)